In [1]:
import pandas as pd 
import numpy as np


In [2]:
book_df=pd.read_csv("collaborative_books_df.csv")

In [3]:
book_df.head()

,Unnamed: 0,title,book_id,user_id_mapping,book_id_mapping,Predicted Rating,Actual Rating
0,0,I Am the Messenger,19057,1537,299,4.5,5
1,1,I Am the Messenger,19057,23039,299,4.9,3
2,2,I Am the Messenger,19057,39096,299,3.9,3
3,3,I Am the Messenger,19057,14631,299,4.7,4
4,4,I Am the Messenger,19057,32816,299,4.3,5


In [4]:
book_df.shape

(196296, 7)

In [23]:
x = book_df.groupby('user_id_mapping').count()["Actual Rating"] >10
good_readers = x[x].index


In [24]:
filtered_rating = book_df[book_df['user_id_mapping'].isin(good_readers)]


In [25]:
y = filtered_rating.groupby('title').count()['Actual Rating']>=5


In [26]:
famous_books = y[y].index


In [27]:
final_ratings = filtered_rating[filtered_rating['title'].isin(famous_books)]


In [28]:
pt = final_ratings.pivot_table(index='title',columns='user_id_mapping',values='Actual Rating')


In [29]:
pt.fillna(0,inplace=True)
pt

user_id_mapping,208,227,315,337,349,427,495,541,548,875,...,53784,54849,55667,57310,58950,59244,62043,63130,64133,69119
title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Child Called It Dave Pelzer 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Christmas Carol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
A Clash of Kings A Song of Ice and Fire 2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Court of Thorns and Roses A Court of Thorns and Roses 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
White Oleander,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wicked The Life and Times of the Wicked Witch of the West The Wicked Years 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WinniethePooh WinniethePooh 1,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
from scipy.sparse import csr_matrix

In [34]:
book_sparse = csr_matrix(pt)

In [36]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [37]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [39]:
distance, suggestion = model.kneighbors(pt.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )

In [40]:
distance

array([[0.        , 8.1240384 , 8.1240384 , 8.30662386, 8.30662386,
        8.66025404]])

In [41]:
suggestion

array([[237, 285, 241,  74, 123,  73]], dtype=int64)

In [42]:
pt.iloc[241,:]

user_id_mapping
208      0.0
227      0.0
315      0.0
337      0.0
349      0.0
        ... 
59244    0.0
62043    0.0
63130    0.0
64133    0.0
69119    0.0
Name: The Mists of Avalon Avalon 1, Length: 293, dtype: float64

In [46]:
for i in range(len(suggestion)):
    print(pt.index[suggestion[i]])

Index(['The Magicians The Magicians 1', 'The Wind in the Willows',
       'The Mists of Avalon Avalon 1', 'Fifty Shades Freed Fifty Shades 3',
       'Lolita', 'Fifty Shades Darker Fifty Shades 2'],
      dtype='object', name='title')


In [48]:
pt.index[3]


'A Clash of Kings A Song of Ice and Fire 2'

In [55]:
book_id = np.where(pt.index =='Sharp Objects')

In [56]:
book_id

(array([], dtype=int64),)

In [51]:
def recommend_book(book_name):
    book_id = np.where(pt.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(pt.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            books = pt.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

In [52]:
recommend_book('The Wind in the Willows')

You searched 'The Wind in the Willows'

The suggestion books are: 

Geralds Game
Lolita
Fifty Shades Freed Fifty Shades 3
The Magicians The Magicians 1
Fifty Shades Darker Fifty Shades 2


In [59]:
recommend_book("The Magicians The Magicians 1")

You searched 'The Magicians The Magicians 1'

The suggestion books are: 

The Wind in the Willows
The Mists of Avalon Avalon 1
Fifty Shades Freed Fifty Shades 3
Lolita
Fifty Shades Darker Fifty Shades 2


In [58]:
book_df.title.unique(

array(['I Am the Messenger',
       'The Name of the Wind The Kingkiller Chronicle 1',
       'We Should All Be Feminists', 'Betrayed House of Night 2',
       'The Paris Wife',
       'Anna and the French Kiss Anna and the French Kiss 1',
       'Not That Kind of Girl A Young Woman Tells You What Shes Learned',
       'Under the Dome', 'This Lullaby',
       'Shiver The Wolves of Mercy Falls 1', 'The Art of War',
       'Unbroken A World War II Story of Survival Resilience and Redemption',
       'From Dead to Worse Sookie Stackhouse 8',
       'Good Omens The Nice and Accurate Prophecies of Agnes Nutter Witch',
       'The Five People You Meet in Heaven', 'Slammed Slammed 1',
       'The Coincidence of Callie Kayden The Coincidence 1',
       'The Girl Who Played with Fire Millennium 2',
       'Lean In Women Work and the Will to Lead', 'Faefever Fever 3',
       'Pretty Little Liars Pretty Little Liars 1', 'Shutter Island',
       'The Darkest Minds The Darkest Minds 1',
       'Jur